In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src/')
import dataset_prep

Import de la base de données

In [174]:
selected_columns = [
"nb_packets_sent",
"distance",
"distRealSR1",
"pos_x_send",
"pos_y_send",
"spd_x_send",
"spd_y_send",
"sendTime",
"receiver",
"sender",
"label"
]

In [175]:
df = dataset_prep.import_dataset('../data/DoS_0709_new_columns.csv', selected_columns)

In [173]:
df.columns

Index(['sendTime', 'sender', 'receiver', 'pos_x_send', 'pos_y_send',
       'spd_x_send', 'spd_y_send', 'label', 'pos_x_rec', 'pos_y_rec',
       'pos_x_rec_f', 'pos_y_rec_f', 'nb_packets_sent', 'distRealSR1',
       'distance'],
      dtype='object')

In [176]:
dataset_prep.clean_dataset(df)

Nombre de lignes avant nettoyage :  4753433
Nombre de lignes après nettoyage :  4753433


Analyse et vérification des données

In [177]:
senders = np.unique(df["sender"].values)
senders

array([    9,    15,    21, ..., 24399, 24405, 24411], dtype=int64)

In [178]:
print(len(senders))

4067


In [179]:
df.loc[df['sender'] == senders[0]].sort_values("sendTime")

,sendTime,sender,receiver,pos_x_send,pos_y_send,spd_x_send,spd_y_send,label,nb_packets_sent,distRealSR1,distance
4,25212.602877,9,45,257.604242,141.917136,-2.409908,14.316189,0,96,7.936029,96.038156
5,25213.602877,9,45,255.242943,156.206787,-2.566640,14.210027,0,96,13.296912,107.233302
6,25214.602878,9,45,252.857751,170.495740,-2.342829,14.254332,0,96,15.193227,116.162309
7,25215.602879,9,45,250.525544,184.832952,-2.352874,14.315002,0,96,19.588409,123.083379
8,25216.602880,9,45,248.511212,199.166902,-2.079440,14.360857,0,96,20.325683,127.877519
...,...,...,...,...,...,...,...,...,...,...,...
94,25291.602880,9,225,134.821280,1024.506204,-0.602227,8.137469,0,96,3.264490,77.948582
65,25291.602880,9,147,134.821280,1024.506204,-0.602227,8.137469,0,96,12.772639,126.355556
91,25292.602869,9,219,134.539820,1030.197090,-0.270067,3.653434,0,96,7.918098,15.731521
95,25292.602869,9,225,134.539820,1030.197090,-0.270067,3.653434,0,96,4.261582,66.586594


In [180]:
# Copie de la bdd pour récuperer les colonnes sans impacter la vraie base de données

data_copy = df.copy()
data_copy = data_copy.drop(["label"], axis=1)

columns = data_copy.columns

Amélioration à faire :

Séléctionner seulement les sender avec value_counts > 20

Séparation de la base de données : diviser pour régner

In [181]:
# /!\ attention à bien utiliser la base de données rangée par sender pour limiter la perte de données
# Indice égal à 0, 1 ou 2
# Célia = 0
# Chléo = 1
# Thomas = 2

def split_data_in_3(df, indice):
    # Division de la bdd en 3
    split_data = np.array_split(df,3)

    # Division en plusieurs sous-bdd pour diminuer le temps de calcul par exécution
    #divided_data = np.array_split(split_data[indice],50)

    return split_data

/!\ à bien changer l'indice !!

In [182]:
divided_dataframes = split_data_in_3(df.sort_values("sender"),0)

In [11]:
len(divided_dataframes)

3

In [183]:
sequence = (df.loc[df['sender'] == senders[0]].sort_values("sendTime"))[0:20]
sequence.values.tolist()

[[25212.60287667693,
  9.0,
  45.0,
  257.6042419688774,
  141.91713555892144,
  -2.409908154165353,
  14.316189069287113,
  0.0,
  96.0,
  7.936029105711839,
  96.03815620967256],
 [25213.60287742229,
  9.0,
  45.0,
  255.24294286856767,
  156.2067871239683,
  -2.566640456502577,
  14.210026600469446,
  0.0,
  96.0,
  13.29691234588276,
  107.23330168824552],
 [25214.602878220827,
  9.0,
  45.0,
  252.8577507018196,
  170.495740224512,
  -2.3428285879092865,
  14.254332296090103,
  0.0,
  96.0,
  15.193226622630618,
  116.16230917509078],
 [25215.602879059785,
  9.0,
  45.0,
  250.5255438289604,
  184.83295234553023,
  -2.35287357908116,
  14.315001990352824,
  0.0,
  96.0,
  19.58840934243324,
  123.08337872534791],
 [25216.602879924656,
  9.0,
  45.0,
  248.51121218164064,
  199.1669019594416,
  -2.0794399698786794,
  14.360856696518791,
  0.0,
  96.0,
  20.32568340556122,
  127.8775190955432],
 [25217.602880801045,
  9.0,
  45.0,
  246.6122355719212,
  213.4859322360176,
  -2.10868

In [184]:
df.loc[df['receiver'] == senders[0]]['label'].tolist()[0]

0

Création des séquences

In [327]:
def sequence_creation(df):
    senders_sequences = []
    senders_label = []
    for sender in senders:
        sender_data_sorted = df.loc[df['sender'] == sender].sort_values("sendTime")
        sender_data_sorted = sender_data_sorted.drop(["label","sender"], axis=1)
        sequence_array = []

        length = df.loc[df['receiver'] == sender].shape[0]
        slide = 10
        start = 0
        end = 20

        # On récupère la valeur du label pour ce sender
        label = df.loc[df['receiver'] == senders[0]]['label'].tolist()[0]
        if (length > 20):
            senders_label.append(label)

        # On vérifie qu'il est possible de faire une séquence de taille 20
        while length > 20:
            # Extraction par tranche de 20 avec une inter de 10
            sequence = sender_data_sorted[start:end]

            # Labels correspondant
            #labels =  pd.Series.tolist(sequence["label"])

            # On transforme les 13 en 1, cette formule marche toujours si on met d'autres types d'attaques
            #labels[:] = [x if x == 0 else 1 for x in labels]

            # Attribution des tableaux numpy
            sequence_array.append(sequence.values.tolist())
            #label_array.append(labels)

            # Mise à jour des variables
            start += slide
            end += slide
            length -= 20
        
        if len(sequence_array)>0:
            senders_sequences.append(sequence_array)

    return senders_sequences, senders_label

/!\ à bien changer l'indice !!

In [328]:
sequence_test, label_test = sequence_creation(divided_dataframes[0])

Vérification des séquences obtenues

In [333]:
len(sequence_test)

1702

Test RNN

In [188]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers, applications, utils, models, optimizers, Input, callbacks
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [334]:
# Transformation en array numpy
X = sequence_test
y = np.array(label_test)

In [335]:
# Séparation en données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [336]:
# print("X_train : ", X_train.shape)
print("y_train : ", y_train.shape)
# print("X_test : ", X_test.shape)
print("y_test : ", y_test.shape)

y_train :  (1531,)
y_test :  (171,)


In [337]:
X_flat = [item for sitem in X_train for item in sitem]
# print(X_flat[:20])


In [385]:
# X_train = np.asarray(X_train[:1000])
X_train = np.asarray(X_flat[:1000])
y_train = np.asarray(y_train[:1000])

In [395]:
#TODO Certaines sequences ne font pas 20 de longueur, sequence creation à refaire

len(X_flat[0])

20

In [341]:
i=0
for item in X_flat:
    if len(item) != 20:
        print('FAUX',i)
        i += 1
    else:
        print( 'True')

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [409]:
simple_model = keras.Sequential([
    layers.Dense(32,input_shape=(len(X_train[0]),len(X_train[0][0]),), activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [410]:
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [411]:
opt = keras.optimizers.SGD(learning_rate=0.1)
simple_model.compile(loss='mean_squared_error', optimizer='adam', metrics='accuracy')

In [412]:
history = simple_model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=2, validation_split=0.3)

Epoch 1/10
6/6 - 3s - loss: 0.0579 - accuracy: 0.9291 - val_loss: 1.8811e-04 - val_accuracy: 1.0000 - 3s/epoch - 464ms/step
Epoch 2/10
6/6 - 0s - loss: 1.3723e-04 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000 - 357ms/epoch - 60ms/step
Epoch 3/10
6/6 - 0s - loss: 9.9787e-05 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 1.0000 - 297ms/epoch - 50ms/step
Epoch 4/10
6/6 - 0s - loss: 3.3205e-05 - accuracy: 1.0000 - val_loss: 0.0073 - val_accuracy: 1.0000 - 281ms/epoch - 47ms/step
Epoch 5/10
6/6 - 0s - loss: 2.3105e-05 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 1.0000 - 314ms/epoch - 52ms/step
Epoch 6/10
6/6 - 0s - loss: 1.7492e-05 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 1.0000 - 246ms/epoch - 41ms/step
Epoch 7/10
6/6 - 0s - loss: 1.4281e-05 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 1.0000 - 252ms/epoch - 42ms/step
Epoch 8/10
6/6 - 0s - loss: 1.0970e-05 - accuracy: 1.0000 - val_loss: 0.0165 - val_accuracy: 1.0000 - 290ms/epoch - 48ms